# Segmentation en phrases

## Imports

In [3]:
import os
import sys
import nltk
from nltk.tokenize import sent_tokenize

In [4]:
import sys
import nltk
from nltk.tokenize import sent_tokenize

## Fichiers d'inputs et d'outputs

In [5]:
infile = "../tp4/corpus.txt"
outfile = "../tp4/corpusw2v.txt"

## Segmentation en phrases du corpus complet et création d'un nouveau fichier

**Important** : pour traiter le corpus complet, indiquez `LIMIT = None`

In [6]:
LIMIT = None

In [7]:
with open(outfile, 'w', encoding="utf-8") as output:
    with open(infile, encoding="utf-8", errors="backslashreplace") as f:
        content = f.readlines()
        content = content[:LIMIT] if LIMIT is not None else content
        n_lines = len(content)
        for i, line in enumerate(content):
            if i % 100 == 0:
                print(f'processing line {i}/{n_lines}')
            sentences = sent_tokenize(line)
            for sent in sentences:
                output.write(sent + "\n")
print("Done")

processing line 0/1
Done


# Word Embeddings : le modèle Word2Vec

## Imports

In [8]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [9]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [10]:
infile = f"../tp4/corpusw2v.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [11]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [12]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [13]:
len(bigram_phrases.vocab.keys())

953417

Prenons une clé au hasard :

In [15]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

59_bko


Le dictionnaire indique le score de cette coocurrence :

In [16]:
bigram_phrases.vocab[key_]

1

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [17]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [18]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [19]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [20]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [21]:
print(corpus[:100])

[['deuxieme', 'annee', '.'], ['-', 'n', '.', '130', '.'], ['20_centimes', 'le_numero', 'i', '-', 'liflercrfcdi', '3', 'juin', '1925', ',', 'drapeau', 'telephone', '.'], [':', 'administration', '-', 'publicite', '.', '.', '...', '.', '.', '...', '....,', '16333', '*', 'redaction', '163', '.', '33', '>>', 'redaction', '{', 'apres', '19', 'heures', ')', '296', '.', '89', 'adressa', 'telegraphique', ':', 'drapouge', '-', 'bruxelleo', 'compte', '-', 'cheques_postaux', ';', 'n', '"', '122', '.', '595', 'organe', 'quotidien', 'du_parti', 'communiste', 'beige', '(', 's', 'b', 'i', '57', '-', '5s', 'rue_dea', 'a', '>>', 'eaens', ',', '57', '-', '59', 'bko', '/', 'oellel', '>', '*>,', 'abonnements', ':', 'belgique', ',', '30_septembre', '31', 'decent', '>', 'etranger', '.'], ['1', 'an', '.'], ['i', 't', "'", 'abonne_dans', 'tous_tes', ',', 'bureaux', 'de', 'punis', '.'], ['les', 'dettes_belges', 'envers_les', 'etats_-_unis', 'vers', "l_'_inflation", 'et', 'la_main', '-', 'mise', 'americaine', 's

## Entrainement d'un modèle Word2Vec sur ce corpus

In [22]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=7, # On ignore les mots qui n'apparaissent pas au moins 7 fois dans le corpus
    workers=3, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 3.55 s
Wall time: 6.47 s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [24]:
outfile = f"../tp4/corpusdb.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [25]:
model = Word2Vec.load("../tp4/corpusdb.model")

### Imprimer le vecteur d'un terme

In [27]:
model.wv["banque"]

array([-0.13188373, -0.6026362 ,  0.33602905,  2.333313  ,  0.21708763,
        0.43139032,  0.04926314,  1.820423  ,  0.67257786,  1.4755898 ,
       -0.44795865, -2.3396027 ,  0.54695386, -0.37319398, -1.8555108 ,
        0.38626394, -1.310726  ,  0.7865423 , -0.6655431 ,  2.9735935 ,
       -0.49011585, -0.00877105,  1.8593655 , -3.4466689 ,  3.039669  ,
       -0.3737799 ,  1.617716  , -1.4171431 , -1.0412405 , -1.4924067 ,
       -0.01603176, -0.4810704 ], dtype=float32)

In [28]:
model.wv["etat"]

array([ 0.4951036 , -0.23177837, -1.0529529 ,  0.37591457, -0.24939391,
       -1.6867355 , -0.26483667,  0.30665636, -0.79198337,  1.8225222 ,
        0.32028905, -0.84671676,  0.20436318, -0.02616134, -0.00405181,
       -0.05192585,  0.12048884,  0.400746  ,  1.1258572 ,  0.17232727,
        0.7332141 ,  1.3255436 ,  1.4142447 , -0.5056767 ,  0.20474449,
       -0.3070956 , -0.58283293, -0.8261282 ,  0.2706182 ,  0.43015268,
       -0.05444039,  0.28174654], dtype=float32)

### Calculer la similarité entre deux termes

In [30]:
model.wv.similarity("droit", "democratie")

0.80686194

In [31]:
model.wv.similarity("police", "violence")

0.89107156

In [32]:
model.wv.similarity("police", "ordre")

0.82525754

In [33]:
model.wv.similarity("asile", "invasion")

0.71591127

In [34]:
model.wv.similarity("droit", "democratie")

0.80686194

### Chercher les mots les plus proches d'un terme donné

In [35]:
model.wv.most_similar("noir", topn=10)

[('bleu', 0.9553493857383728),
 ('rose', 0.9520599842071533),
 ('jaune', 0.9505562782287598),
 ('transvaal', 0.9497373700141907),
 ('renard', 0.9476518630981445),
 ('fll', 0.9424518346786499),
 ('aoot', 0.9422782063484192),
 ('home', 0.9420903921127319),
 ('pectoral', 0.9413021206855774),
 ('star', 0.9405019879341125)]

In [36]:
model.wv.most_similar("soir", topn=10)

[('matin', 0.9528449773788452),
 ('siege', 0.9342163801193237),
 ('drame', 0.9251589179039001),
 ('courrier', 0.9147465229034424),
 ('roi', 0.9114739894866943),
 ('cinema', 0.9073225855827332),
 ('siecle', 0.9067301154136658),
 ('cou', 0.9067167043685913),
 ('frere', 0.9047124981880188),
 ('30_novembre', 0.9034886360168457)]

In [38]:
model.wv.most_similar("actions", topn=10)

[('jouissance', 0.842002809047699),
 ('actions_privilegiees', 0.840703010559082),
 ('depots', 0.8149512410163879),
 ('obligations', 0.8100811839103699),
 ('liberees', 0.806624174118042),
 ('banques', 0.8015186786651611),
 ('caisse', 0.7845936417579651),
 ('ateliers', 0.7776685953140259),
 ('parts', 0.7759085893630981),
 ('ordinaires', 0.7643119096755981)]

In [39]:
model.wv.most_similar("femme", topn=10)

[('histoire', 0.977387011051178),
 ('digne', 0.9680662751197815),
 ('geste', 0.9646246433258057),
 ('combinaison', 0.9639960527420044),
 ('langage', 0.9629129767417908),
 ('admirable', 0.9615954160690308),
 ('puissance', 0.9596827030181885),
 ('vive', 0.9592717289924622),
 ('grace', 0.9591938853263855),
 ('de_mme_coilomb', 0.9587934613227844)]

In [40]:
model.wv.most_similar("homme", topn=10)

[('le_roi', 0.9521457552909851),
 ('seul', 0.9495663046836853),
 ('assure', 0.9463509321212769),
 ('aussitot', 0.9461931586265564),
 ('grave', 0.9457772374153137),
 ('connu', 0.9414288997650146),
 ('prononce', 0.9413462281227112),
 ("l_'_auteur", 0.9397786259651184),
 ('une_autre', 0.9384989142417908),
 ('vraiment', 0.9377130270004272)]

In [41]:
model.wv.most_similar("blanc", topn=10)

[('rouge', 0.863639771938324),
 ('paysan', 0.8543131947517395),
 ('matadero_extra', 0.8497834205627441),
 ('coq', 0.8409463167190552),
 ('lapin', 0.8383168578147888),
 ('village', 0.836881697177887),
 ('mier', 0.8360007405281067),
 ('tambour', 0.8342134952545166),
 ('chateau', 0.8339943885803223),
 ('jaune', 0.8303341269493103)]

In [42]:
model.wv.most_similar("metro", topn=10)

[('rio', 0.942065417766571),
 ('union_parisienne', 0.9187070727348328),
 (';_royal_dutch', 0.9066250324249268),
 ('cana', 0.8796939849853516),
 ('changhal', 0.8671905994415283),
 ('052', 0.8597162365913391),
 (';_central_mining', 0.8539825677871704),
 (':_petroflna', 0.8537185192108154),
 ('inchange', 0.8504599928855896),
 ('contro', 0.8498464226722717)]

In [43]:
model.wv.most_similar("train", topn=10)

[('fleuve', 0.9530004262924194),
 ('legataire', 0.9464680552482605),
 ('fermier', 0.9438720345497131),
 ('match', 0.9436465501785278),
 ('foyer', 0.9390085935592651),
 ('triomphe', 0.9384762644767761),
 ('fauteuil', 0.9363344311714172),
 ('battu', 0.9348222017288208),
 ("d_'_aviation", 0.9345833659172058),
 ('situe', 0.9336017966270447)]

In [45]:
model.wv.most_similar("football", topn=10)

[('debt', 0.9482006430625916),
 ('bank', 0.9465391039848328),
 ('aie', 0.9450823068618774),
 ('ttt', 0.9399576783180237),
 ('of_the', 0.9390336275100708),
 ('station', 0.9388034343719482),
 ('cu', 0.9384173154830933),
 ('dei', 0.9373806715011597),
 ('by_the', 0.9328418374061584),
 ('Y=', 0.9320881962776184)]

In [46]:
model.wv.most_similar("droit", topn=10)

[('cause', 0.9711174368858337),
 ('rendre', 0.9488545060157776),
 ('succes', 0.9430756568908691),
 ('maniere', 0.9414774775505066),
 ('nature', 0.9409788250923157),
 ('mettre', 0.9372210502624512),
 ('pris', 0.9348628520965576),
 ('pouvoir', 0.9339888691902161),
 ('maintenir', 0.9283112287521362),
 ('trouver', 0.927722156047821)]

### Faire des recherches complexes à travers l'espace vectoriel

In [48]:
print(model.wv.most_similar(positive=['republicains', 'democrates'], negative=['politiciens']))

[('fait_durant', 0.8948108553886414), ('pondant', 0.8918882608413696), ('supportent', 0.8902180790901184), ('multiplier', 0.8834531903266907), ('pelerins', 0.8819226622581482), ('volumes', 0.8780476450920105), ('expositions', 0.8767145872116089), ('voiles', 0.8754528164863586), ('pates', 0.873284637928009), ('instituts', 0.870937705039978)]


In [49]:
print(model.wv.most_similar(positive=['eau', 'pluie'], negative=['mer']))

[('overpelt_l', 0.842007040977478), ('activite_economique', 0.8355211615562439), ('"_gr', 0.8323809504508972), ('boom_p', 0.8175777792930603), ('lh', 0.8036455512046814), ('wr', 0.8031070232391357), ('kownocap', 0.801292359828949), ('lite', 0.8003257513046265), ('wilsele_jouis', 0.7995355725288391), ('**_ur', 0.7994494438171387)]


In [50]:
print(model.wv.most_similar(positive=['noir', 'blanc'], negative=['couleur']))

[('oct', 0.854617714881897), ('danube', 0.8488488793373108), ('mara', 0.8435136675834656), ('plau', 0.830959677696228), ('cmb', 0.8236958384513855), ('emb', 0.8222048878669739), ('livr', 0.8221510648727417), ('bombay', 0.8182555437088013), ('lin_am', 0.8067922592163086), ('allem', 0.8059899806976318)]


In [51]:
print(model.wv.most_similar(positive=['technologie', 'metier'], negative=['communication']))

KeyError: "Key 'technologie' not present in vocabulary"

In [ ]:
print(model.wv.most_similar(positive=['danse', 'cluture'], negative=['tango']))

[('soumissions_aura_lieu', 0.7645087242126465), ('fut_consacree', 0.7476798892021179), ('foire_agricole', 0.747564435005188), ('sera_accessible', 0.746957004070282), ('notariales', 0.7426692247390747), ('hsures', 0.7381367683410645), ('nitive', 0.7352807521820068), ('mars_1013', 0.7328065037727356), ('fetes_seront_organisees', 0.7259426116943359), ('la_salle_marugg', 0.7248245477676392)]
